<a href="https://colab.research.google.com/github/cindysotton/Video-Game-Prediction/blob/main/Scrapping_jeuxvideo_com.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import des librairies
from mpl_toolkits.mplot3d import Axes3D
import numpy as np 
import os 
import pandas as pd 
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as bs
import requests

# on récupère la page d'un jeu comme base
url_jv = 'https://www.jeuxvideo.com/jeux/jeu-1289839/'

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"}
webpage = Request(url=url_jv, headers=headers) 
page_jv = urlopen(webpage).read()
print(page_jv)
soup = bs(page_jv, "html.parser")

In [ ]:
#Déclaration des listes
gname = []
description = []
date_sortie_fr = []
notes = []
publisher = []
studio = []
genre = []
themes = []
modele_eco = []

In [ ]:
def get_info(link):
    page = requests.get(link)
    soup = bs(page.text, "lxml")
    try : 
      if soup.find('h1', attrs = {'class' : "breadcrumb__item"}).text :
        gname = soup.find('h1', attrs = {'class' : "breadcrumb__item"}).text
    except : 
        gname = "np.nan"
    try : 
      if soup.find('p', attrs = {'class' : "gameCharacteristicsMain__synopsis"}).text :
        description = soup.find('p', attrs = {'class' : "gameCharacteristicsMain__synopsis"}).text
    except : 
        description = "np.nan"
    try : 
      if soup.find('div', string="Sortie France").find_next_sibling("div").span.text :
        date_sortie_fr = soup.find('div', string="Sortie France").find_next_sibling("div").span.text.strip()
    except : 
        date_sortie_fr = "np.nan"
    try : 
      if soup.find('div', string="Editeur(s) / Développeur(s)").find_next_sibling("div").span.text :
        publisher = soup.find('div', string="Editeur(s) / Développeur(s)").find_next_sibling("div").span.text.strip()
    except : 
        publisher = "np.nan"    
    try : 
      if soup.find('div', string="Editeur(s) / Développeur(s)").find_next_sibling("div").text :
        studio = soup.find('div', string="Editeur(s) / Développeur(s)").find_next_sibling("div").text.strip()
    except : 
        studio = "np.nan"
    try : 
      if soup.find('div', string="Genre(s)").find_next_sibling("div").text :
        genre = soup.find('div', string="Genre(s)").find_next_sibling("div").text.strip()
    except : 
        genre = "np.nan"   
    try : 
      if soup.find('div', string="Thème(s)").find_next_sibling("div").text :
        themes = soup.find('div', string="Thème(s)").find_next_sibling("div").text.strip()
    except : 
        themes = "np.nan"      
    try : 
      if soup.find('div', string="Modèle(s) économique(s)").find_next_sibling("div").text :
        modele_eco = soup.find('div', string="Modèle(s) économique(s)").find_next_sibling("div").text.strip()
    except : 
        modele_eco = "np.nan"
    try : 
      if soup.find('div', attrs = {'class' : "userOpinionsPerPlatform"}).text :
        notes = soup.find('div', attrs = {'class' : "userOpinionsPerPlatform"}).text.strip()
    except : 
        notes = "np.nan"       

    line = [gname,description,date_sortie_fr,publisher,studio,genre,themes,modele_eco,notes]
    return line

In [ ]:
genre_indice_list = []
for i in range (2000,3000):
    all_jeu = "https://www.jeuxvideo.com/tous-les-jeux/genre-"+str(i)+'/'
    page = requests.get(all_jeu)
    soup = bs(page.text,'html5lib')
    error = soup.find('h1', string="Vous êtes sur une page qui n'existe plus ou n'a jamais existé.")
    if error is None:
      genre_indice_list.append(str(i))

In [ ]:
jeu_list = []
for genre in genre_indice_list :
  for i in range(1,1000):
      all_jeu = "https://www.jeuxvideo.com/tous-les-jeux/genre-"+genre+'/?p='+str(i)
      page = requests.get(all_jeu)
      soup = bs(page.text,'lxml')
      a = soup.find_all('span',class_="gameTitleLink__196nPy")
      divs = soup.find_all('div',class_="container__3Ow3zD")
      for div in divs :
        a = div.find("a")
        if ("/jeux/jeu" in a["href"]) and (a["href"] not in jeu_list):
          jeu_list.append(a["href"])

In [ ]:
base = "https://www.jeuxvideo.com"
df = pd.DataFrame(columns = ['Nom','Description','Date de sortie FR','Editeur','Studio','Genres','Thèmes','Modèle économique'])

for jeu in jeu_list :
    row = get_info(base+jeu)
    a = pd.DataFrame([row],columns = ['Nom','Description','Date de sortie FR','Editeur','Studio','Genres','Thèmes','Modèle économique'])
    df = pd.concat([df,a],ignore_index=True)

df = df.drop_duplicates().reset_index(drop=True)

,Nom,Description,Date de sortie FR,Editeur,Studio,Genres,Thèmes,Modèle économique
0,Atomic Heart,Atomic Heart est un jeu d'aventure orienté FPS...,\n 21 févr....,\n Mundfish...,\n\n Mundfi...,\n\n Av...,np.nan,\n\n Payant...
1,Call of Duty : Modern Warfare 2 (2022),Version reboot de l'emblématique jeu de tir à ...,\n 28 oct. ...,\n Acti...,\n\n Ac...,\n\n FP...,np.nan,np.nan
2,Dead Space (2023),Dead Space Remake est une version réinterprété...,\n 27 janv....,\n Motive S...,\n\n Motive...,\n\n Av...,np.nan,np.nan
3,The Day Before,The Day Before est un MMORPG d'action/aventure...,\n 10 nov. ...,\n MYTONA\n...,\n\n MYTONA...,\n\n MM...,np.nan,np.nan
4,Call of Duty : Warzone 2.0,Call of Duty : Warzone 2.0 est un jeu de type ...,\n 16 nov. ...,\n Acti...,\n\n Ac...,\n\n FP...,np.nan,np.nan
5,Overwatch 2,Overwatch 2 est la suite du Hero Shooter de Bl...,\n 04 oct. ...,\n Bliz...,\n\n Bl...,\n\n FP...,np.nan,np.nan
6,Redfall,Redfall est un FPS en open-world se déroulant ...,\n 02 mai 2...,\n Arkane A...,\n\n Arkane...,\n\n FP...,np.nan,np.nan
7,Suicide Squad: Kill the Justice League,Suicide Squad : Kill the Justice League est un...,\n 26 mai 2...,\n Rockstea...,\n\n Rockst...,\n\n Av...,\n\n Su...,np.nan
8,Destiny 2 : Eclipse,np.nan,\n 2023\n ...,\n Bung...,\n\n Bu...,\n\n MM...,np.nan,np.nan
9,Wanted : Dead,Wanted : Dead est un mélange de shooter et hac...,\n 14 févr....,\n 110 Indu...,\n\n 110 In...,\n\n FP...,np.nan,np.nan
